In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

app_key = os.getenv("APP_KEY")
app_secret = os.getenv("APP_SECRET")

In [15]:
from alibabacloud_dingtalk.oauth2_1_0.client import Client as DingtalkClient
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_dingtalk.oauth2_1_0 import models as dingtalk_oauth_models

config = open_api_models.Config()
config.protocol = "https"
config.region_id = "central"
dingtalk_client = DingtalkClient(config)

get_access_token_request = dingtalk_oauth_models.GetAccessTokenRequest(
    app_key=app_key, app_secret=app_secret
)
try:
    dingtalk_client.get_access_token(get_access_token_request)
except Exception as err:
    raise err